In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'ctr_valid_inference_mixtral.pkl'
with open(inference_path, 'rb') as f:
    inference_dict = pickle.load(f)
print(len(inference_dict))

6040


In [3]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [4]:
user_valid_movie_rating_df = ratings_df.groupby('user_id').nth(-2)
user_valid_movie_rating_df.head(10)

,user_id,movie_id,rating
51,1,1566,4
452,10,2693,5
528,100,1208,2
612,1000,3363,5
989,1001,3756,2
1055,1002,2352,5
1084,1003,281,5
1565,1004,3804,2
1657,1005,1466,3
1701,1006,2706,4


In [7]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 1
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 4624 -- No: 1318 -- Invalid: 98
invalid_users: [1164, 1283, 1375, 1413, 1431, 1468, 147, 1533, 1584, 1606, 1612, 1638, 1645, 1679, 1710, 1725, 1735, 1755, 1772, 1844, 1854, 1948, 1955, 2024, 2077, 2118, 2121, 2302, 2375, 2415, 2501, 2604, 2608, 2624, 2635, 2695, 276, 2798, 2839, 3265, 330, 3395, 3397, 3538, 3586, 3716, 3762, 3782, 3842, 3867, 387, 3929, 3942, 4120, 4167, 4206, 4291, 431, 435, 4478, 4526, 4539, 4543, 4596, 4626, 4627, 4703, 4727, 4792, 4794, 486, 4981, 5108, 5109, 511, 5207, 5265, 541, 5561, 5574, 5649, 5651, 5687, 572, 5747, 5748, 5759, 5774, 5775, 5786, 5788, 5900, 5946, 5980, 671, 882, 978, 98]
6040


In [8]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user_id'] == user]['rating'].values[0]
    # print(original_rating)
    if original_rating >= 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[ 410  643]
 [ 908 4079]]
AUC: 0.6036451659404669
Precision: 0.8638288860652266
Recall: 0.817926609183878
Accuracy: 0.7432119205298013


In [13]:
movies_df = pd.read_csv('../movie_summary_25_words.csv')
movies_df[movies_df.movie_id == 1466]

,movie_id,name,cleaned_genre,year,summary
1439,1466,Donnie Brasco,"Crime,Drama",1997,"""Donnie Brasco"" is a gripping crime drama base..."


In [9]:
ctr_valid_dataset_path = './ctr_zeroshot_dataset/ctr_valid_zeroshot_dataset.pkl'
with open(ctr_valid_dataset_path, 'rb') as f:
    ctr_valid_data_dict = pickle.load(f)
print(len(ctr_valid_data_dict))

6040


In [14]:
print(ctr_valid_data_dict[1005])


    You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
    The user is a male lawyer between 35 to 44 years of age
            The user has a diverse taste in movies, with a preference for those that are thought-provoking and emotionally impactful. They enjoy movies that explore complex themes and blur the lines between reality and fantasy, as seen in their appreciation for "Being John Malkovich" and "Bowfinger." The user also has a soft spot for classic Christmas films, as evidenced by their fondness for "Miracle on 34th Street."
            
            In terms of genres, the user leans towards drama and comedy, with a particular interest in coming-of-age stories and character-driven narratives. They are drawn to movies that feature strong performances and explore the complexities of human relationships, as seen in their appreciation for "Diner" and "Schi

In [19]:
for user in invalid_users:
    print(user, inference_dict[user])

1164 
To analyze the user's behavior and predict whether they will like "Who Framed Roger Rabbit?" or not, we need to consider their movie watching history and preferences.

The user tends to enjoy movies that are character-driven, thought-provoking, and emotionally resonant. They have a clear preference for romantic comedies, comedies with wit and humor, and dramas that explore complex themes such as self-discovery, relationships, and the human condition.

Looking at the user's movie watching history, we can see that they have enjoyed movies such as "Mickey Blue Eyes," "The Mighty," "Out of Africa," "Rushmore," "Sleepy Hollow," "The Talented Mr. Ripley," "The Thomas Crown Affair," "Velvet Goldmine," and "Where the Heart Is." These movies all share common themes of love, relationships, and personal growth, with a strong focus on character development and storytelling.

On the other hand, the user has disliked movies such as "Midsummer Night's Dream," "Mystery Men," "Pushing Tin," "Stir